### PoC Nutrition asked queries :

**Table of content :** <br>
[1️⃣ | Nombre de produits par nutriscore. Quel est le pourcentage de représentation associé?](#1) <br>
[2️⃣ | Les 4 valeurs possibles du Nova_Group](#2) <br>
[3️⃣ | Quel est la moyenne des fibres par nutriscore ?](#3) <br>
[4️⃣ | Quels sont le TOP10 des pnns_groups_2 qui comportent le plus d'additifs](#4) <br>
[5️⃣ | Afficher le taux de sucre/fat moyen et médian des produits avec un nutriscore A ou B](#5) <br>
[6️⃣ | Combien avons-nous de produits qui ont un taux de carbohydrates > aux sucres](#6) <br>

In [35]:
import psycopg2 as pgconn
import pandas as pd

print('⬆️ This contains imports & pg connector.')

# Init db connection and open cursor
def db_connect(cred):
    return pgconn.connect(
                host=cred['host'],
                database=cred['database'],
                user=cred['user'],
                password=cred['password']
            )

credentials = {
            'host': 'localhost',
            'database': 'open_food_data_db',
            'user': 'postgres',
            'password': ''
        }

conn = db_connect(credentials)
cur = conn.cursor()

⬆️ This contains imports & pg connector.


> ***Nota Bene*** <br>
> Les résultats obtenus ne sont pas sur l'ensemble des données. <br>
> Les queries ont été rédigées dans une syntaxe compatible à postgres.

-----
#### 1️⃣ | Nombre de produits par nutriscore. Quel est le pourcentage de représentation associé? <a name=1></a>

```sql
SELECT
  nutriscore_score,
  cnt,
  ROUND(100.0 * (( cnt) / (sum(cnt) OVER ())), 2) AS percentage
FROM (
	SELECT nutriscore_score, 
	count(*) AS cnt
	FROM records
	WHERE nutriscore_score::FLOAT IS NOT NULL 
	GROUP BY nutriscore_score
	ORDER BY cnt DESC
) x;
```

In [32]:
cur.execute('SELECT nutriscore_score, cnt, ROUND(100.0 * (( cnt) / (sum(cnt) OVER ())), 2) AS percentage FROM ( SELECT nutriscore_score,  count(*) AS cnt FROM records WHERE nutriscore_score::FLOAT IS NOT NULL  GROUP BY nutriscore_score ORDER BY cnt DESC ) x;')
res = cur.fetchall()

print('\nNombre de produits par nutriscore et % associés : \n')
df = pd.DataFrame(res, columns=['nutriscore_score', 'n count', '%'])
print(df.to_string(index=False))


Nombre de produits par nutriscore et % associés : 

nutriscore_score  n count    %
            13.0       26 8.50
            11.0       22 7.19
             2.0       20 6.54
            17.0       18 5.88
             3.0       18 5.88
            14.0       16 5.23
             1.0       16 5.23
             0.0       14 4.58
            16.0       14 4.58
            12.0       14 4.58
             5.0       14 4.58
             6.0       14 4.58
            20.0       12 3.92
            21.0       10 3.27
            18.0        8 2.61
            19.0        8 2.61
            15.0        8 2.61
            -2.0        6 1.96
             4.0        6 1.96
            23.0        4 1.31
            22.0        4 1.31
            26.0        4 1.31
             9.0        4 1.31
            25.0        4 1.31
             8.0        2 0.65
            10.0        2 0.65
            28.0        2 0.65
            -5.0        2 0.65
            29.0        2 0.65
            27.0 

-------
#### 2️⃣ | Les 4 valeurs possibles du Nova_Group <a name=2></a>

```sql
SELECT DISTINCT(nova_group) FROM records ORDER BY nova_group DESC;
```

In [42]:
cur.execute('SELECT DISTINCT(nova_group) FROM records ORDER BY nova_group DESC;')
res = cur.fetchall()

print('\nLes 4 valeurs possibles du `nova_group` :\n')
df = pd.DataFrame(res, columns=['Nova group values'])
print(df.to_string(index=False))


Les 4 valeurs possibles du `nova_group` :

Nova group values
             None
              4.0
              3.0
              2.0
              1.0


---------
#### 3️⃣ | Quel est la moyenne des fibres par nutriscore ? <a name=3></a>

```sql
SELECT
	nutriscore_score,
	AVG(fiber_100g::FLOAT) as avg_fiber_per_100g
FROM records
WHERE nutriscore_score::FLOAT IS NOT NULL 
GROUP BY nutriscore_score
ORDER BY avg_fiber_per_100g DESC
```

In [17]:
cur.execute('SELECT nutriscore_score, AVG(fiber_100g::FLOAT) as avg_fiber_per_100g FROM records WHERE nutriscore_score::FLOAT IS NOT NULL GROUP BY nutriscore_score ORDER BY avg_fiber_per_100g DESC')
res = cur.fetchall()

print('\nMoyenne des fibres par nutriscore :\n')
df = pd.DataFrame(res, columns=['Nutriscore', 'Fibre pour 100g']).sort_values(by='Nutriscore')
print(df.to_string(index=False))


Moyenne des fibres par nutriscore :

Nutriscore  Fibre pour 100g
      -2.0         1.800000
      -3.0              NaN
      -4.0         1.600000
      -5.0              NaN
       0.0         1.750000
       1.0         0.900000
      10.0         3.500000
      11.0         2.514125
      12.0         1.280000
      13.0         2.100000
      14.0         1.966667
      15.0              NaN
      16.0         1.000000
      17.0         2.400000
      18.0         0.800000
      19.0         1.900000
       2.0         1.119250
      20.0         0.000000
      21.0              NaN
      22.0              NaN
      23.0         1.300000
      24.0              NaN
      25.0         1.550000
      26.0              NaN
      27.0              NaN
      28.0              NaN
      29.0              NaN
       3.0         3.650000
      36.0         0.000000
       4.0              NaN
       5.0         1.350000
       6.0         4.500000
       7.0              NaN
       8.0

-----
#### 4️⃣ | Quels sont le TOP10 des pnns_groups_2 qui comportent le plus d'additifs <a name='4'></a>

```sql
SELECT
	pnns_groups_2,
	AVG(additives_n::FLOAT) as additives_n
FROM records
GROUP BY pnns_groups_2
ORDER BY additives_n DESC
LIMIT 10;
```

In [16]:
cur.execute('SELECT pnns_groups_2, AVG(additives_n::FLOAT) as additives_n FROM records GROUP BY pnns_groups_2 ORDER BY additives_n DESC LIMIT 10;')
res = cur.fetchall()

print('\nQuels sont le TOP10 des pnns_groups_2 qui comportent le plus d\'additifs\n')
df = pd.DataFrame(res, columns=['pnns_group_2', 'N additifs'])
print(df.to_string(index=False))


Quels sont le TOP10 des pnns_groups_2 qui comportent le plus d'additifs

          pnns_group_2  N additifs
          Fruit juices         NaN
                  Nuts         NaN
                  Meat         NaN
        Processed meat         NaN
Pizza pies and quiches    8.000000
        Dairy desserts    4.000000
            Appetizers    4.000000
    Biscuits and cakes    3.785714
                  Fats    3.500000
                Sweets    3.111111


------
#### 5️⃣ | Afficher le taux de sucre/fat moyen et médian des produits avec un nutriscore A ou B <a name=5></a>

```sql
SELECT
    nutriscore_grade,
	AVG(sugars_100g::FLOAT) avg_sugar,
	AVG(fat_100g::FLOAT) avg_fat,
	(SELECT percentile_disc(0.5) WITHIN GROUP (order by records.sugars_100g) FROM records) as med_sugar,
	(SELECT percentile_disc(0.5) WITHIN GROUP (order by records.fat_100g) FROM records) as med_fat
FROM records
WHERE nutriscore_grade = 'a' OR nutriscore_grade = 'b'
GROUP BY nutriscore_grade
```

In [15]:
cur.execute('SELECT nutriscore_grade, AVG(sugars_100g::FLOAT) avg_sugar, AVG(fat_100g::FLOAT) avg_fat, (SELECT percentile_disc(0.5) WITHIN GROUP (order by records.sugars_100g) FROM records) as med_sugar, (SELECT percentile_disc(0.5) WITHIN GROUP (order by records.fat_100g) FROM records) as med_fat FROM records WHERE nutriscore_grade = \'a\' OR nutriscore_grade = \'b\' GROUP BY nutriscore_grade')
res = cur.fetchall()

print('\nTaux moyen/medin de sucre/fat pour les produits avec un nutriscore A ou B\n')
df = pd.DataFrame(res, columns=['nutriscore', 'avg surgar', 'avg fat', 'med sugar', 'med fat'])
print(df.to_string(index=False))


Taux moyen/medin de sucre/fat pour les produits avec un nutriscore A ou B

nutriscore  avg surgar  avg fat med sugar med fat
         a    6.018333 4.183333       2.7    22.3
         b    8.386417 5.984583       2.7    22.3


----
#### 6️⃣ | Combien avons-nous de produits qui ont un taux de carbohydrates > aux sucres <a name=6></a>

```sql
SELECT
	COUNT(DISTINCT(code))
FROM records
WHERE carbohydrates_100g::FLOAT > sugars_100g::FLOAT
````

In [25]:
cur.execute('SELECT COUNT(DISTINCT(code)) FROM records WHERE carbohydrates_100g::FLOAT > sugars_100g::FLOAT')
res = cur.fetchall()

print(f'\nNombre de produits ou le taux de carbohydrates > sucres : {res[0][0]}\n')


Nombre de produits ou le taux de carbohydrates > sucres : 457

